In [ ]:
# init

# Sparse Matrix Operations

In [ ]:
"""
Suppose we have very large sparse vectors, which contains a lot of
zeros and double .

find a data structure to store them
get the dot product of them
"""

In [ ]:
#! /usr/bin/env python3

def vector_to_index_value_list(vector):
    return [(i, v) for i, v in enumerate(vector) if v != 0.0]


def dot_product(iv_list1, iv_list2):

    product = 0
    p1 = len(iv_list1) - 1
    p2 = len(iv_list2) - 1

    while p1 >= 0 and p2 >= 0:
        i1, v1 = iv_list1[p1]
        i2, v2 = iv_list2[p2]

        if i1 < i2:
            p1 -= 1
        elif i2 < i1:
            p2 -= 1
        else:
            product += v1 * v2
            p1 -= 1
            p2 -= 1

    return product


def __test_simple():
    print(dot_product(vector_to_index_value_list([1., 2., 3.]),
                      vector_to_index_value_list([0., 2., 2.])))
    # 10


def __test_time():
    vector_length = 1024
    vector_count = 1024
    nozero_counut = 10

    def random_vector():
        import random
        vector = [0 for _ in range(vector_length)]
        for i in random.sample(range(vector_length), nozero_counut):
            vector[i] = random.random()
        return vector

    vectors = [random_vector() for _ in range(vector_count)]
    iv_lists = [vector_to_index_value_list(vector) for vector in vectors]

    import time

    time_start = time.time()
    for i in range(vector_count):
        for j in range(i):
            dot_product(iv_lists[i], iv_lists[j])
    time_end = time.time()

    print(time_end - time_start, 'seconds')


if __name__ == '__main__':
    __test_simple()
    __test_time()


## Sparse matrix Multiplication

In [ ]:
"""
Given two sparse matrices A and B, return the result of AB.

You may assume that A's column number is equal to B's row number.

Example:

A = [
  [ 1, 0, 0],
  [-1, 0, 3]
]

B = [
  [ 7, 0, 0 ],
  [ 0, 0, 0 ],
  [ 0, 0, 1 ]
]


     |  1 0 0 |   | 7 0 0 |   |  7 0 0 |
AB = | -1 0 3 | x | 0 0 0 | = | -7 0 3 |
                  | 0 0 1 |
"""

#### Solution 1: Basic Multiplication with Zero Check

In [ ]:
# Python solution without table (~156ms):
def multiply(self, a, b):
    """
    Multiply two matrices A and B, skipping zero elements.
    
    :param a: List[List[int]] - matrix A
    :param b: List[List[int]] - matrix B
    :return: List[List[int]] - result of AB
    """
    if a is None or b is None:
        return None
    m, n, l = len(a), len(b[0]), len(b[0])
    if len(b) != n:
        raise Exception("A's column number must be equal to B's row number.")
    
    # Initialize result matrix with zeros
    c = [[0 for _ in range(l)] for _ in range(m)]
    
    # Multiply matrices and skip zero elements
    for i, row in enumerate(a):
        for k, eleA in enumerate(row):
            if eleA:
                for j, eleB in enumerate(b[k]):
                    if eleB:
                        c[i][j] += eleA * eleB
    return c


#### Solution 2: Optimized with Table for B

In [ ]:
# Python solution with only one table for B (~196ms):
def multiply(self, a, b):
    """
    Multiply two matrices A and B, using a lookup table for B.
    
    :param a: List[List[int]] - matrix A
    :param b: List[List[int]] - matrix B
    :return: List[List[int]] - result of AB
    """
    if a is None or b is None:
        return None
    m, n, l = len(a), len(a[0]), len(b[0])
    if len(b) != n:
        raise Exception("A's column number must be equal to B's row number.")
    
    # Initialize result matrix and table for B
    c = [[0 for _ in range(l)] for _ in range(m)]
    table_b = {}
    
    # Store non-zero elements of B in a lookup table
    for k, row in enumerate(b):
        table_b[k] = {}
        for j, eleB in enumerate(row):
            if eleB:
                table_b[k][j] = eleB

    # Multiply using the lookup table for B
    for i, row in enumerate(a):
        for k, eleA in enumerate(row):
            if eleA:
                for j, eleB in table_b[k].items():
                    c[i][j] += eleA * eleB
    return c


#### Solution 3: Optimized with Tables for A and B

In [ ]:
# Python solution with two tables (~196ms):
def multiply(self, a, b):
    """
    Multiply two matrices A and B, using lookup tables for both A and B.
    
    :param a: List[List[int]] - matrix A
    :param b: List[List[int]] - matrix B
    :return: List[List[int]] - result of AB
    """
    if a is None or b is None:
        return None
    m, n = len(a), len(b[0])
    if len(b) != n:
        raise Exception("A's column number must be equal to B's row number.")
    
    l = len(b[0])
    table_a, table_b = {}, {}

    # Store non-zero elements of A in a lookup table
    for i, row in enumerate(a):
        for j, ele in enumerate(row):
            if ele:
                if i not in table_a:
                    table_a[i] = {}
                table_a[i][j] = ele

    # Store non-zero elements of B in a lookup table
    for i, row in enumerate(b):
        for j, ele in enumerate(row):
            if ele:
                if i not in table_b:
                    table_b[i] = {}
                table_b[i][j] = ele

    # Initialize result matrix
    c = [[0 for j in range(l)] for i in range(m)]

    # Multiply using the lookup tables for both A and B
    for i in table_a:
        for k in table_a[i]:
            if k not in table_b:
                continue
            for j in table_b[k]:
                c[i][j] += table_a[i][k] * table_b[k][j]
    return c
